# "nltk를 이용한 Naive Bayes 분석"
> "네이버 영화 네티즌·관람객 평점 리뷰 데이터"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [NaiveBayes]
- hide: false
- search_exclude: true

## 1. 개요
---
- **작성일** : 2020-10-11
- **작성자** : 채연희
- **분석 목적** : `nltk`의 `Naive Bayes` 패키지를 이용해 긍정적 문장, 부정적 문장 분류
- **분석 대상** : 네이버 영화 네티즌·관람객 평점 리뷰 데이터

<br/>  
- **사용 라이브러리**
```
# 데이터 수집
from urllib.request import urlopen
from bs4 import BeautifulSoup
　
# 데이터 처리 및 정제
import re
import pandas as pd
　
# 한국어 형태소 분석 / 토큰화 및 나이브베이즈
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
import nltk
　
# 혼동행렬
from sklearn.metrics import confusion_matrix, classification_report
```
  
<br/>  

## 2. 데이터 수집 및 정제
---
### 2.1 데이터 수집
- `urllib`, `BeautifulSoup`를 이용해 https://movie.naver.com/movie/point/af/list.nhn 에서 2020-10-11 기준으로 1페이지 ~ 500페이지 수집  
(데이터 정제 이후 `부정` 데이터가 적어 500 ~ 1000 페이지의 `부정` 데이터만 추가 수집)
- 제목, 평점, 리뷰 데이터를 `Data.Frame`으로 정렬

<br/>  
### 2.2 데이터 정제
- 평점은 있으나 리뷰가 없는 데이터 삭제
- 평점 `5` 미만은 `부정`, `6` 이상은 `긍정`으로 치환
- 평점 `4 ~ 7` 사이의 데이터는 긍·부정이 동시에 나타나는 경우가 많아 문맥에 따라 재분류
- 그 외 ① 영화와 관련 없는 리뷰 ② 긍·부정 판단이 모호한 리뷰 ③ 스토리, 소재 등만 나열한 리뷰 ④ 반어법을 사용한 리뷰 등은 삭제

<br/>  
- `부정` 데이터 추가 수집 이후 순서 랜덤 정렬
- 최종 데이터 : [movie_data_set.csv](https://github.com/chaeyeonhee/Naver-movie-review/blob/main/movie_data_set.csv)
  
<br/>  

## 3. 데이터 불러오기
---

### 3.1 데이터 조회

- `총 4935`건 데이터. `긍정 2932`건, `부정 2003`건

In [ ]:
import pandas as pd
movie_data_set = pd.read_csv("movie_data_set.csv")
movie_data_set

,title,point,review,type
0,만추,10,하..좋다.. 역시 지난 영화는 지나고 나이를 먹고 또 봐야 느끼는 감정이 달라지...,긍정
1,천문: 하늘에 묻는다,10,음...두분에 연기에 가슴이 뛰었습니다.,긍정
2,나를 차버린 스파이,2,평점이 왜이리 높아개노잼 20분 보다 더는못참고 끕니다,부정
3,캐리비안의 해적: 죽은 자는 말이 없다,8,집에서 보기 좋은 영화예요~^^,긍정
4,언힌지드,1,그냥 5만원정도 주면서까지 영화 보라고 해도 시간이 아까울 정도입니다,부정
...,...,...,...,...
4930,링컨 차를 타는 변호사,8,"미국식 쿨함이 그런 것인가, 다들 참 복잡한 세상 단순하게 사는 것 같아 부럽기도 ...",긍정
4931,인사이드 잡,1,재미도 없지만 볼 가치도 없는 다큐.,부정
4932,뮬란,1,뮬란 본연의 매력은 다버리고 무협 판타지로 만들어버림,부정
4933,패스트 & 퓨리어스 - 도쿄 드리프트,9,드리프트 액션 진짜 볼만함,긍정


In [ ]:
movie_data_set[movie_data_set.type == '긍정']

,title,point,review,type
0,만추,10,하..좋다.. 역시 지난 영화는 지나고 나이를 먹고 또 봐야 느끼는 감정이 달라지...,긍정
1,천문: 하늘에 묻는다,10,음...두분에 연기에 가슴이 뛰었습니다.,긍정
3,캐리비안의 해적: 죽은 자는 말이 없다,8,집에서 보기 좋은 영화예요~^^,긍정
6,국제수사,10,곽도원 김희원! 제가 좋아하는 배우분들이라 믿고 봤는데 잼나용,긍정
9,담보,10,소이가 너무 귀여워요ㅠㅠ 그리고 많이 울었어요.추천합니다~,긍정
...,...,...,...,...
4928,콘 에어,10,요새 어지간한 헐리우드 영화보다 낫네ㅎ,긍정
4929,담보,10,성동일이라서 뭔가 더 집중할수있었던 영화웃다가 울다가 마지막엔 오열 후회없는 영화 ...,긍정
4930,링컨 차를 타는 변호사,8,"미국식 쿨함이 그런 것인가, 다들 참 복잡한 세상 단순하게 사는 것 같아 부럽기도 ...",긍정
4933,패스트 & 퓨리어스 - 도쿄 드리프트,9,드리프트 액션 진짜 볼만함,긍정


In [ ]:
movie_data_set[movie_data_set.type == '부정']

,title,point,review,type
2,나를 차버린 스파이,2,평점이 왜이리 높아개노잼 20분 보다 더는못참고 끕니다,부정
4,언힌지드,1,그냥 5만원정도 주면서까지 영화 보라고 해도 시간이 아까울 정도입니다,부정
5,죽지않는 인간들의 밤,1,알바의 존재를 이제야 알았네요...도망가세여,부정
7,안시성,1,걍 다 재미없고 끔찍한 장면만 담아낸듯한 영화. 끔찍하게 할거면 그 수준에 맞는 재...,부정
8,그것만이 내 세상,1,제발 좀 영화 장르좀 확실해라!!! 코미디를 보러왔는데 정작 본건 억지감동물이냐!!...,부정
...,...,...,...,...
4921,나우 유 씨 미 2,2,최악. 중국인 감독이라 그런지 1편에 비해 너무나도 과한 액션과 지루한 스토리.마술...,부정
4923,죽지않는 인간들의 밤,1,이거 블라인드 시사회때 보고 진짜 주위 사람들한테 보지 말라고 홍보하고 싶었음......,부정
4925,담보,2,너무나 예측 가능한 스토리 전개와 억지스러운 요소들이 빛났던 영화. 코로나때문에 작...,부정
4931,인사이드 잡,1,재미도 없지만 볼 가치도 없는 다큐.,부정


  
<br/>  

### 3.2 학습 및 테스트 데이터 나누기

In [ ]:
# 테스트용 데이터
movie_test = movie_data_set.iloc[:1000,:]
movie_test

,title,point,review,type
0,만추,10,하..좋다.. 역시 지난 영화는 지나고 나이를 먹고 또 봐야 느끼는 감정이 달라지...,긍정
1,천문: 하늘에 묻는다,10,음...두분에 연기에 가슴이 뛰었습니다.,긍정
2,나를 차버린 스파이,2,평점이 왜이리 높아개노잼 20분 보다 더는못참고 끕니다,부정
3,캐리비안의 해적: 죽은 자는 말이 없다,8,집에서 보기 좋은 영화예요~^^,긍정
4,언힌지드,1,그냥 5만원정도 주면서까지 영화 보라고 해도 시간이 아까울 정도입니다,부정
...,...,...,...,...
995,증인,1,candle(apkj****)-좋은 영화? 고소? 어디서 미친 OOO가 멍멍 짖는구...,부정
996,시,4,너무 루즈하압니드아아,부정
997,파수꾼,9,가해자의 탈을 쓴 기태가 점차 관계의 어긋남으로 위태로워지는 그 감정선의 변화가 정...,긍정
998,담보,1,10글자 평도 아까운 진부한 신파,부정


In [ ]:
# 훈련용 데이터
movie_train = movie_data_set.iloc[1000:,:]
movie_train

,title,point,review,type
1000,너의 이름은.,10,레전드영화 설명이필요없다,긍정
1001,82년생 김지영,1,절대공감되지않는이야기 판타지적인이야기와 피해의식이라는정신병이만들어낸 결과물 이영화를...,부정
1002,테넷,10,놀란은 천재다...,긍정
1003,기생충,1,개인적으로 기대했던것 보단 별로.. 스페인영화 제목이... 암튼 그 영화에서 부분적...,부정
1004,"색, 계",8,연기에서 진심으로 변화하는 과정의 섬세한 묘사. 탕웨이가 본인과 동료들의 목숨까지 ...,긍정
...,...,...,...,...
4930,링컨 차를 타는 변호사,8,"미국식 쿨함이 그런 것인가, 다들 참 복잡한 세상 단순하게 사는 것 같아 부럽기도 ...",긍정
4931,인사이드 잡,1,재미도 없지만 볼 가치도 없는 다큐.,부정
4932,뮬란,1,뮬란 본연의 매력은 다버리고 무협 판타지로 만들어버림,부정
4933,패스트 & 퓨리어스 - 도쿄 드리프트,9,드리프트 액션 진짜 볼만함,긍정


  
<br/>  

## 4. 나이브베이즈 실행
---

### 4.1 패키지 설치 및 import

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download("all")

In [ ]:
!pip install konlpy

  
<br/>  

### 4.2 학습 데이터 토큰화 및 나이브베이즈 실행

In [ ]:
from konlpy.tag import Okt
pos_tagger = Okt()

train_data = movie_train[['review','type']].values.tolist()

def tokenize(doc):
  return ['/'.join(i) for i in pos_tagger.pos(doc)]

train_doc = [(tokenize(i[0]),i[1]) for i in train_data]
tokens = [j for i in train_doc for j in i[0]]

def term_exists(doc):
  return {word : (word in set(doc)) for word in tokens}

x_train = [(term_exists(d), c) for d, c in train_doc]

classifier = nltk.NaiveBayesClassifier.train(x_train)
classifier.show_most_informative_features()

Most Informative Features
           아깝다/Adjective = True               부정 : 긍정     =     27.7 : 1.0
              뭔/Modifier = True               부정 : 긍정     =     22.3 : 1.0
                쓰레기/Noun = True               부정 : 긍정     =     20.4 : 1.0
                 가슴/Noun = True               긍정 : 부정     =     17.6 : 1.0
                봤어요/Verb = True               긍정 : 부정     =     16.7 : 1.0
          지루하고/Adjective = True               부정 : 긍정     =     16.2 : 1.0
                마세요/Verb = True               부정 : 긍정     =     15.5 : 1.0
            안됨/Adjective = True               부정 : 긍정     =     15.2 : 1.0
                 성도/Noun = True               부정 : 긍정     =     14.1 : 1.0
                 노잼/Noun = True               부정 : 긍정     =     13.6 : 1.0


  
<br/>  

### 4.3 텍스트 분류 확인
- 데이터셋에 없는 텍스트 입력

In [ ]:
test = [('어린 승이 연기한 배우 너무 귀엽더라 ㅎㅎ 가족들과 보기에 딱 좋은 힐링무비~~')]
test_doc = tokenize(test[0])

test_f = {word:(word in tokens) for word in test_doc}
classifier.classify(test_f)

'긍정'

In [ ]:
test = [('좋은 배우 써 놓고 내용은 정말 하타치입니다..웃음 포인트 감동 포인트 하나 없습니다')]
test_doc = tokenize(test[0])

test_f = {word:(word in tokens) for word in test_doc}
classifier.classify(test_f)

'부정'

In [ ]:
test = [('곽도원 때문에 예매해서봣는데,,그냥 유치뽕짝,,10대들이나좋아할듯,, 내가본영화중 시간버린 기억에남을작품')]
test_doc = tokenize(test[0])

test_f = {word:(word in tokens) for word in test_doc}
classifier.classify(test_f)

'부정'

In [ ]:
test = [('오드리햅번 넘 사랑스럽고 아름다움~♡영화에 나오는 명소들도 하나하나씩 다시금 추억이 떠오르고 보는 내내 설레이는 영화')]
test_doc = tokenize(test[0])

test_f = {word:(word in tokens) for word in test_doc}
classifier.classify(test_f)

'긍정'

  
<br/>  

### 4.3 테스트 데이터 분류

In [ ]:
test_result = []

for i in movie_test.review:
  test = [(i)]
  test_doc = tokenize(test[0])
  test_f = {word:(word in tokens) for word in test_doc}
  result = classifier.classify(test_f)
  test_result.append(result)

  
<br/>  

## 5. 혼동행렬 조회
---
- `test_actual` : 테스트 데이터의 실제 `type`  
`test_predict` : 모델이 예측한 테스트 데이터의 `type`
- 정확도 94%로 유의하게 도출

In [ ]:
test_actual = pd.Series(movie_test.type, name = 'actual')
test_predict = pd.Series(test_result, name = 'predict')

pd.crosstab(test_actual, test_predict)

predict,긍정,부정
actual,,
긍정,509,20
부정,44,427


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(confusion_matrix(test_actual, test_predict))

[[509  20]
 [ 44 427]]


In [ ]:
print(classification_report(test_actual, test_predict))

              precision    recall  f1-score   support

          긍정       0.92      0.96      0.94       529
          부정       0.96      0.91      0.93       471

    accuracy                           0.94      1000
   macro avg       0.94      0.93      0.94      1000
weighted avg       0.94      0.94      0.94      1000



  
<br/>  

## 6. 일치하지 않는 데이터 확인
---
- 오탈자가 있거나 비표준어, 초성 등을 사용한 경우 / 비유적 표현을 사용한 경우 등으로 확인
- 정제 작업을 보다 치밀하게 할 경우 결과가 향상될 것으로 예상됨.

In [ ]:
movie_test[movie_test.type != test_predict]

,title,point,review,type
15,국제수사,6,좀더 장황하고 긴당감잇게 연출되럿스면,부정
30,브리짓 존스의 일기 - 열정과 애정,4,1탄을 너무 재밌게봤는데 그렇게 매력있던 브리짓을 이렇게 매력없게 만들수있나 싶다....,부정
32,"그대, 고맙소 : 김호중 생애 첫 팬미팅 무비",1,ㄴㅈ이네요ㄴㅈ이에요,부정
34,애프터: 그 후,4,기대가 컸나봐요 ...,부정
66,해수의 아이,1,2020년에 개봉한 영화가 맞는지 제가 지금 몇년도에 살고있는지 날짜의 소중함을 알...,부정
...,...,...,...,...
986,그린랜드,10,혜성 충돌 재난영화는 처음이라 기대하지 않았는데 영화 2010 스케일과 맞먹는 영...,긍정
991,마마,8,빗나간 모성애..그래도 나는 나름 재밌게 봤다. 주인공들 연기도 쩔었고 귀신캐릭터도...,긍정
992,용서는 없다,8,근데 국내최고의 부검의가 토막난시체가 서로 다른사람이라는걸 모르나? 우짰든 힘있는위...,긍정
996,시,4,너무 루즈하압니드아아,부정
